# Предсказание покупки следующего курса

### Загрузим необходимые библиотеки

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

### Загружаем датасеты

In [400]:
df = pd.read_csv("./content/train_dataset_train.csv")
dft = pd.read_csv("./content/test_dataset_test.csv")

### и объединяем их

In [401]:
len_df = len(df)
dfres = pd.concat([df, dft], ignore_index=True)
#len(dfres)
#dfres = dfres.fillna(0)
#dfres.info(verbose=True)

In [402]:
dfres = dfres.fillna(-1)

### Рассмотрим датасеты по ближе

In [403]:
df.head(3)

,id,age_indicator,month_id,student_id,program_id,carts_created_at,spent_time_total,spent_time_to_complete_hw,completed_hw,failed_hw,...,p_total_calls,p_was_conversations,p_total_duration,support_feedback_avg,feedback_avg_d1,feedback_avg_d2,feedback_avg_d3,feedback_avg_d4,feedback_avg_d5,target
0,15182,32.0,9/30/2020,6694527,1469,8/26/2020,163.0,0.0,1.0,0.0,...,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,0
1,89385,NaN,6/30/2021,6712877,1392,8/5/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,47931,NaN,2/28/2021,6659444,376,6/20/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


```
 0   id                         200000 non-null  int64   # уникальное значение каждой строк
 1   age_indicator              159123 non-null  float64 # возраст в условных единицах
 2   month_id                   200000 non-null  object  # последний день месяца среза
 3   student_id                 200000 non-null  int64   # id студента  
 4   program_id                 200000 non-null  int64   # id продукта
 5   carts_created_at           200000 non-null  object  # дата создания заявки
 6   spent_time_total           86309 non-null   float64 # время за учебой в условных единицах
 7   spent_time_to_complete_hw  42467 non-null   float64 # время, потраченное на выполнение дз
 8   completed_hw               97599 non-null   float64 # кол-во выполненных дз
 9   failed_hw                  97599 non-null   float64 # кол-во проваленных дз 
 10  reworked_hw                97599 non-null   float64 # кол-во переделанных дз 
 11  interacted_hw              97599 non-null   float64 # кол-во дз, которыми занимался студент в этом месяце
 12  avg_hw_mark                44496 non-null   float64 # средняя оценка за дз
 13  test_with_good_mark        97599 non-null   float64 # кол-во тестов с удовлетворительной оценкой
 14  test_with_great_mark       97599 non-null   float64 # кол-во тестов с отличной оценкой
 15  webinars                   97599 non-null   float64 # кол-во просмотренных вебинаров
 16  avg_quiz_result            16259 non-null   float64 # средняя оценка за квизы
 17  notes                      97599 non-null   float64 # кол-во оставленных заметок
 18  hw_leader                  97599 non-null   float64 # был лидером в групповых заданиях
 19  lessons                    97599 non-null   float64 # кол-во уроков, которые открывал пользователь
 20  activity                   97599 non-null   float64 # общая активность студента в личном кабинете
 21  bought_d1                  200000 non-null  int64   # кол-во прошлых покупок в направлении 1
 22  bought_d2                  200000 non-null  int64   
 23  bought_d3                  200000 non-null  int64  
 24  bought_d4                  200000 non-null  int64  
 25  bought_d5                  200000 non-null  int64  
 26  bought_avg_duration        10359 non-null   float64
 27  payment_type               200000 non-null  int64   # тип оплаты
 28  promo                      200000 non-null  object  # участвовал ли в любой промокампании
 29  price                      200000 non-null  float64 # цена программы
 30  communication_type         198595 non-null  object  # тип создания заявки (ОП или сам через сайт)
 31  auto_payment               200000 non-null  int64   # автопокупка (1 или 0)
 32  ABC                        200000 non-null  object  # сегмент клиента по готовности купить
 33  city                       13592 non-null   object  # город пользователя
 34  country                    12944 non-null   object  # страна пользователя 
 35  gender                     199940 non-null  float64 # пол
 36  speed_recall               21880 non-null   float64 # скорость перезвона клиенту (1 - быстро, 7 - долго)
 37  os                         77374 non-null   object  # операционная система пользователя
 38  browser                    77374 non-null   object  # название браузера 
 39  platform                   77361 non-null   object  # платформа
 40  m_avg_talk_duration        51537 non-null   float64 # средняя длительность разговора
 41  m_avg_duration             45764 non-null   float64 # средняя длительность звонка 
 42  m_missed_calls             51537 non-null   float64 # кол-во пропущенных звонков
 43  m_total_calls              51537 non-null   float64 # всего звонков
 44  m_was_conversations        51537 non-null   float64 # кол-во звонков, когда состоялся диалог
 45  m_total_duration           51537 non-null   float64 # длительность всего
 46  p_avg_talk_duration        16453 non-null   float64
 47  p_avg_duration             13488 non-null   float64
 48  p_missed_calls             16453 non-null   float64
 49  p_total_calls              16453 non-null   float64
 50  p_was_conversations        16453 non-null   float64
 51  p_total_duration           16453 non-null   float64
 52  support_feedback_avg       37557 non-null   float64 # средняя удовлетворенность службой поддержки за год
 53  feedback_avg_d1            48496 non-null   float64 # средний балл по курсам направления 1
 54  feedback_avg_d2            14805 non-null   float64
 55  feedback_avg_d3            17778 non-null   float64
 56  feedback_avg_d4            25076 non-null   float64
 57  feedback_avg_d5            15733 non-null   float64
 58  target                     200000 non-null  int64  
``` 

### Приспособы для one_hot_encoding

In [404]:
def convert_to_dict(df):
    res = dict()
    i = 0
    for el in df:
        if el == -1:
            continue
        if el not in res:
            res[el] = i
            i += 1
    return res

In [405]:
def simple_one_hot_encoding(df, col_name):
    codes = convert_to_dict(df[col_name])
        
    new_columns = {}
    for i in range(len(codes)):
        new_columns[col_name+str(i)] = [0 for i in range(len(df))]
    new_columns = pd.DataFrame(new_columns, index=df.index)
    new_columns = new_columns.astype(np.int8)
    df = pd.concat([df, new_columns], axis=1)

    for i in range(len(df)):
        code = df.iloc[i, df.columns.get_loc(col_name)]
        if code != -1:
            ccode = codes[code]
            df.iloc[i, df.columns.get_loc(col_name+str(ccode))] = 1
    return df, codes

In [406]:
def digital_convert_to_dict(vmax):
    res = dict()
    
    i = 0
    j = 0
    while i <= vmax:
        if i < 10:
            res[j] = i
            i += 1
            j += 1
        elif i < 100:
            res[j] = i
            i += 10
            j += 1
        elif i < 1000:
            res[j] = i
            i += 100
            j += 1
        elif i < 10000:
            res[j] = i
            i += 1000
            j += 1
        elif i < 100000:
            res[j] = i
            i += 1000
            j += 1
        elif i < 1000000:
            res[j] = i
            i += 10000
            j += 1
    return res

In [407]:
def digital_conv(code, codes):
    if code == -1:
        return None
    prev_c = 0
    for c in codes:
        if code >= codes[c]:
            prev_c = c
        else:
            break
    return prev_c

In [408]:
def digital_one_hot_encoding(df, col_name):
    
    vmax = df[col_name].max()
    
    codes = digital_convert_to_dict(vmax)
        
    new_columns = {}
    for code in range(len(codes)):
        new_columns[col_name+str(code)] = [0 for i in range(len(df))]
    new_columns = pd.DataFrame(new_columns, index=df.index)
    new_columns = new_columns.astype(np.int8)
    df = pd.concat([df, new_columns], axis=1)

    for i in range(len(df)):
        code = df.iloc[i, df.columns.get_loc(col_name)]
        ccode = digital_conv(code, codes)
        if ccode != None:
            df.iloc[i, df.columns.get_loc(col_name+str(ccode))] = 1
    return df, codes

### анализируем данные

In [409]:
# кандидат на one_hoting_encode
d = dfres['age_indicator'].value_counts()
l = len(d)
rmax = dfres['age_indicator'].max()
rmin = dfres['age_indicator'].min()
d, l, rmax, rmin

(-1.0     58326
  33.0     5767
  25.0     5759
  35.0     5742
  15.0     5742
  21.0     5739
  30.0     5737
  39.0     5732
  2.0      5730
  10.0     5727
  13.0     5726
  4.0      5725
  9.0      5724
  8.0      5724
  28.0     5724
  36.0     5723
  38.0     5722
  1.0      5721
  16.0     5715
  22.0     5711
  31.0     5711
  11.0     5711
  29.0     5709
  32.0     5703
  6.0      5701
  27.0     5701
  12.0     5700
  23.0     5699
  7.0      5699
  19.0     5698
  37.0     5689
  3.0      5688
  17.0     5686
  24.0     5676
  26.0     5669
  14.0     5664
  20.0     5652
  34.0     5626
  5.0      5620
  18.0     5232
  40.0     4547
 Name: age_indicator, dtype: int64,
 41,
 40.0,
 -1.0)

In [410]:
dfres, codes_age_indicator = digital_one_hot_encoding(dfres, 'age_indicator')
dfres = dfres.drop('age_indicator', axis=1)

In [411]:
# кандидат на one_hoting_encode
d = dfres['month_id'].value_counts()
l = len(d)
rmax = dfres['month_id'].max()
rmin = dfres['month_id'].min()
d, l, rmax, rmin

(11/30/2021    34088
 10/31/2021    31019
 9/30/2021     28439
 8/31/2021     25913
 7/31/2021     23513
 6/30/2021     21552
 5/31/2021     19651
 4/30/2021     17936
 3/31/2021     16170
 2/28/2021     14298
 1/31/2021     12680
 12/31/2020    10857
 11/30/2020     8986
 10/31/2020     7276
 9/30/2020      5792
 8/31/2020      3844
 7/31/2020      2211
 6/30/2020       772
 Name: month_id, dtype: int64,
 18,
 '9/30/2021',
 '1/31/2021')

In [ ]:
dfres, codes_month_id = simple_one_hot_encoding(dfres, 'month_id')
dfres = dfres.drop('month_id', axis=1)

In [ ]:
# кандидат на one_hoting_encode
d = dfres['program_id'].value_counts()
l = len(d)
rmax = dfres['program_id'].max()
rmin = dfres['program_id'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_program_id = simple_one_hot_encoding(dfres, 'program_id')
dfres = dfres.drop('program_id', axis=1)

In [ ]:
# считаем, что дата создания заявки не очень важна
d = dfres['carts_created_at'].value_counts()
l = len(d)
rmax = dfres['carts_created_at'].max()
rmin = dfres['carts_created_at'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_carts_created_at = simple_one_hot_encoding(dfres, 'carts_created_at')
dfres = dfres.drop('carts_created_at', axis=1)

In [ ]:
d = dfres['spent_time_total'].value_counts()
l = len(d)
rmax = dfres['spent_time_total'].max()
rmin = dfres['spent_time_total'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_spent_time_total = digital_one_hot_encoding(dfres, 'spent_time_total')
dfres = dfres.drop('spent_time_total', axis=1)

In [ ]:
# игнорим
d = dfres['spent_time_to_complete_hw'].value_counts()
l = len(d)
rmax = dfres['spent_time_to_complete_hw'].max()
rmin = dfres['spent_time_to_complete_hw'].min()
d, l, rmax, rmin

In [ ]:
#dfres, codes_program_id = digital_one_hot_encoding(dfres, 'spent_time_to_complete_hw')
dfres = dfres.drop('spent_time_to_complete_hw', axis=1)

In [ ]:
# кандидат на one_hoting_encode
d = dfres['completed_hw'].value_counts()
l = len(d)
rmax = dfres['completed_hw'].max()
rmin = dfres['completed_hw'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_completed_hw = digital_one_hot_encoding(dfres, 'completed_hw')
dfres = dfres.drop('completed_hw', axis=1)

In [ ]:
# разобьем на шаги по 10
d = dfres['failed_hw'].value_counts()
l = len(d)
rmax = dfres['failed_hw'].max()
rmin = dfres['failed_hw'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_failed_hw = digital_one_hot_encoding(dfres, 'failed_hw')
dfres = dfres.drop('failed_hw', axis=1)

In [ ]:
# разобьем на шаги по 10
d = dfres['reworked_hw'].value_counts()
l = len(d)
rmax = dfres['reworked_hw'].max()
rmin = dfres['reworked_hw'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_reworked_hw = digital_one_hot_encoding(dfres, 'reworked_hw')
dfres = dfres.drop('reworked_hw', axis=1)

In [ ]:
#с шагом 1
d = dfres['interacted_hw'].value_counts()
l = len(d)
rmax = dfres['interacted_hw'].max()
rmin = dfres['interacted_hw'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_interacted_hw = digital_one_hot_encoding(dfres, 'interacted_hw')
dfres = dfres.drop('interacted_hw', axis=1)

In [ ]:
#c шагом 10, max=100
d = dfres['avg_hw_mark'].value_counts()
l = len(d)
rmax = dfres['avg_hw_mark'].max()
rmin = dfres['avg_hw_mark'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_avg_hw_mark = digital_one_hot_encoding(dfres, 'avg_hw_mark')
dfres = dfres.drop('avg_hw_mark', axis=1)

In [ ]:
# с шагом 10
d = dfres['test_with_good_mark'].value_counts()
l = len(d)
rmax = dfres['test_with_good_mark'].max()
rmin = dfres['test_with_good_mark'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_test_with_good_mark = digital_one_hot_encoding(dfres, 'test_with_good_mark')
dfres = dfres.drop('test_with_good_mark', axis=1)

In [ ]:
# с шагом 10
d = dfres['test_with_great_mark'].value_counts()
l = len(d)
rmax = dfres['test_with_great_mark'].max()
rmin = dfres['test_with_great_mark'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_test_with_great_mark = digital_one_hot_encoding(dfres, 'test_with_great_mark')
dfres = dfres.drop('test_with_great_mark', axis=1)

In [ ]:
# с шагом 10
d = dfres['webinars'].value_counts()
l = len(d)
rmax = dfres['webinars'].max()
rmin = dfres['webinars'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_webinars = digital_one_hot_encoding(dfres, 'webinars')
dfres = dfres.drop('webinars', axis=1)

In [ ]:
d = dfres['avg_quiz_result'].value_counts()
l = len(d)
rmax = dfres['avg_quiz_result'].max()
rmin = dfres['avg_quiz_result'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_avg_quiz_result = digital_one_hot_encoding(dfres, 'avg_quiz_result')
dfres = dfres.drop('avg_quiz_result', axis=1)

In [ ]:
# с шагом 100
d = dfres['notes'].value_counts()
l = len(d)
rmax = dfres['notes'].max()
rmin = dfres['notes'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_notes = digital_one_hot_encoding(dfres, 'notes')
dfres = dfres.drop('notes', axis=1)

In [ ]:
# С шагом 10
d = dfres['hw_leader'].value_counts()
l = len(d)
rmax = dfres['hw_leader'].max()
rmin = dfres['hw_leader'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_hw_leader = digital_one_hot_encoding(dfres, 'hw_leader')
dfres = dfres.drop('hw_leader', axis=1)

In [ ]:
d = dfres['lessons'].value_counts()
l = len(d)
rmax = dfres['lessons'].max()
rmin = dfres['lessons'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_lessons = digital_one_hot_encoding(dfres, 'lessons')
dfres = dfres.drop('lessons', axis=1)

In [ ]:
d = dfres['activity'].value_counts()
l = len(d)
rmax = dfres['activity'].max()
rmin = dfres['activity'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_activity = digital_one_hot_encoding(dfres, 'activity')
dfres = dfres.drop('activity', axis=1)

In [ ]:
d = dfres['bought_d1'].value_counts()
l = len(d)
rmax = dfres['bought_d1'].max()
rmin = dfres['bought_d1'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_d1 = digital_one_hot_encoding(dfres, 'bought_d1')
dfres = dfres.drop('bought_d1', axis=1)

In [ ]:
d = dfres['bought_d2'].value_counts()
l = len(d)
rmax = dfres['bought_d2'].max()
rmin = dfres['bought_d2'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_d2 = digital_one_hot_encoding(dfres, 'bought_d2')
dfres = dfres.drop('bought_d2', axis=1)

In [ ]:
d = dfres['bought_d3'].value_counts()
l = len(d)
rmax = dfres['bought_d3'].max()
rmin = dfres['bought_d3'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_d3 = digital_one_hot_encoding(dfres, 'bought_d3')
dfres = dfres.drop('bought_d3', axis=1)

In [ ]:
d = dfres['bought_d4'].value_counts()
l = len(d)
rmax = dfres['bought_d4'].max()
rmin = dfres['bought_d4'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_d4 = digital_one_hot_encoding(dfres, 'bought_d4')
dfres = dfres.drop('bought_d4', axis=1)

In [ ]:
d = dfres['bought_d5'].value_counts()
l = len(d)
rmax = dfres['bought_d5'].max()
rmin = dfres['bought_d5'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_d5 = digital_one_hot_encoding(dfres, 'bought_d5')
dfres = dfres.drop('bought_d5', axis=1)

In [ ]:
d = dfres['bought_avg_duration'].value_counts()
l = len(d)
rmax = dfres['bought_avg_duration'].max()
rmin = dfres['bought_avg_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_bought_avg_duration = digital_one_hot_encoding(dfres, 'bought_avg_duration')
dfres = dfres.drop('bought_avg_duration', axis=1)

In [ ]:
d = dfres['payment_type'].value_counts()
l = len(d)
rmax = dfres['payment_type'].max()
rmin = dfres['payment_type'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_payment_type = simple_one_hot_encoding(dfres, 'payment_type')
dfres = dfres.drop('payment_type', axis=1)

In [ ]:
d = dfres['promo'].value_counts()
l = len(d)
rmax = dfres['promo'].max()
rmin = dfres['promo'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_promo = simple_one_hot_encoding(dfres, 'promo')
dfres = dfres.drop('promo', axis=1)

In [ ]:
d = dfres['price'].value_counts()
l = len(d)
rmax = dfres['price'].max()
rmin = dfres['price'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_price = digital_one_hot_encoding(dfres, 'price')
dfres = dfres.drop('price', axis=1)

In [ ]:
d = dfres['communication_type'].value_counts()
l = len(d)
#rmax = dfres['communication_type'].max()
#rmin = dfres['communication_type'].min()
d, l, #rmax, rmin

In [ ]:
dfres, codes_communication_type = simple_one_hot_encoding(dfres, 'communication_type')
dfres = dfres.drop('communication_type', axis=1)

In [ ]:
d = dfres['auto_payment'].value_counts()
l = len(d)
d, l, #rmax, rmin

In [ ]:
dfres, codes_auto_payment = simple_one_hot_encoding(dfres, 'auto_payment')
dfres = dfres.drop('auto_payment', axis=1)

In [ ]:
d = dfres['ABC'].value_counts()
l = len(d)
d, l

In [ ]:
dfres, codes_ABC = simple_one_hot_encoding(dfres, 'ABC')
dfres = dfres.drop('ABC', axis=1)

In [ ]:
d = dfres['city'].value_counts()
l = len(d)
d, l

In [ ]:
dfres, codes_city = simple_one_hot_encoding(dfres, 'city')
dfres = dfres.drop('city', axis=1)

In [ ]:
d = dfres['country'].value_counts()
l = len(d)
#rmax = dfres['price'].max()
#rmin = dfres['price'].min()
d, l, #rmax, rmin

In [ ]:
dfres, codes_country = simple_one_hot_encoding(dfres, 'country')
dfres = dfres.drop('country', axis=1)

In [ ]:
d = dfres['gender'].value_counts()
l = len(d)
rmax = dfres['gender'].max()
rmin = dfres['gender'].min()
d, l, #rmax, rmin

In [ ]:
dfres, codes_gender = simple_one_hot_encoding(dfres, 'gender')
dfres = dfres.drop('gender', axis=1)

In [ ]:
d = dfres['speed_recall'].value_counts()
l = len(d)
rmax = dfres['speed_recall'].max()
rmin = dfres['speed_recall'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_speed_recall = digital_one_hot_encoding(dfres, 'speed_recall')
dfres = dfres.drop('speed_recall', axis=1)

In [ ]:
d = dfres['os'].value_counts()
l = len(d)
d, l

In [ ]:
dfres, codes_os = simple_one_hot_encoding(dfres, 'os')
dfres = dfres.drop('os', axis=1)

In [ ]:
d = dfres['browser'].value_counts()
l = len(d)
d, l

In [ ]:
dfres, codes_browser = simple_one_hot_encoding(dfres, 'browser')
dfres = dfres.drop('browser', axis=1)

In [ ]:
d = dfres['platform'].value_counts()
l = len(d)
d, l

In [ ]:
dfres, codes_platform = simple_one_hot_encoding(dfres, 'platform')
dfres = dfres.drop('platform', axis=1)

In [ ]:
d = dfres['m_avg_talk_duration'].value_counts()
l = len(d)
rmax = dfres['m_avg_talk_duration'].max()
rmin = dfres['m_avg_talk_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_avg_talk_duration = digital_one_hot_encoding(dfres, 'm_avg_talk_duration')
dfres = dfres.drop('m_avg_talk_duration', axis=1)

In [ ]:
d = dfres['p_avg_talk_duration'].value_counts()
l = len(d)
rmax = dfres['p_avg_talk_duration'].max()
rmin = dfres['p_avg_talk_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_p_avg_talk_duration = digital_one_hot_encoding(dfres, 'p_avg_talk_duration')
dfres = dfres.drop('p_avg_talk_duration', axis=1)

In [ ]:
d = dfres['m_avg_duration'].value_counts()
l = len(d)
rmax = dfres['m_avg_duration'].max()
rmin = dfres['m_avg_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_avg_duration = digital_one_hot_encoding(dfres, 'm_avg_duration')
dfres = dfres.drop('m_avg_duration', axis=1)

In [ ]:
dfres, codes_p_avg_duration = digital_one_hot_encoding(dfres, 'p_avg_duration')
dfres = dfres.drop('p_avg_duration', axis=1)

In [ ]:
d = dfres['m_missed_calls'].value_counts()
l = len(d)
rmax = dfres['m_missed_calls'].max()
rmin = dfres['m_missed_calls'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_missed_calls = digital_one_hot_encoding(dfres, 'm_missed_calls')
dfres = dfres.drop('m_missed_calls', axis=1)

In [ ]:
d = dfres['p_missed_calls'].value_counts()
l = len(d)
rmax = dfres['p_missed_calls'].max()
rmin = dfres['p_missed_calls'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_p_missed_calls = digital_one_hot_encoding(dfres, 'p_missed_calls')
dfres = dfres.drop('p_missed_calls', axis=1)

In [ ]:
d = dfres['m_total_calls'].value_counts()
l = len(d)
rmax = dfres['m_total_calls'].max()
rmin = dfres['m_total_calls'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_total_calls = digital_one_hot_encoding(dfres, 'm_total_calls')
dfres = dfres.drop('m_total_calls', axis=1)

In [ ]:
d = dfres['p_total_calls'].value_counts()
l = len(d)
rmax = dfres['p_total_calls'].max()
rmin = dfres['p_total_calls'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_p_total_calls = digital_one_hot_encoding(dfres, 'p_total_calls')
dfres = dfres.drop('p_total_calls', axis=1)

In [ ]:
d = dfres['m_was_conversations'].value_counts()
l = len(d)
rmax = dfres['m_was_conversations'].max()
rmin = dfres['m_was_conversations'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_was_conversations = digital_one_hot_encoding(dfres, 'm_was_conversations')
dfres = dfres.drop('m_was_conversations', axis=1)

In [ ]:
d = dfres['p_was_conversations'].value_counts()
l = len(d)
rmax = dfres['p_was_conversations'].max()
rmin = dfres['p_was_conversations'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_p_was_conversations = digital_one_hot_encoding(dfres, 'p_was_conversations')
dfres = dfres.drop('p_was_conversations', axis=1)

In [ ]:
d = dfres['m_total_duration'].value_counts()
l = len(d)
rmax = dfres['m_total_duration'].max()
rmin = dfres['m_total_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_m_total_duration = digital_one_hot_encoding(dfres, 'm_total_duration')
dfres = dfres.drop('m_total_duration', axis=1)

In [ ]:
d = dfres['p_total_duration'].value_counts()
l = len(d)
rmax = dfres['p_total_duration'].max()
rmin = dfres['p_total_duration'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_p_total_duration = digital_one_hot_encoding(dfres, 'p_total_duration')
dfres = dfres.drop('p_total_duration', axis=1)

In [ ]:
d = dfres['support_feedback_avg'].value_counts()
l = len(d)
rmax = dfres['support_feedback_avg'].max()
rmin = dfres['support_feedback_avg'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_support_feedback_avg = digital_one_hot_encoding(dfres, 'support_feedback_avg')
dfres = dfres.drop('support_feedback_avg', axis=1)

In [ ]:
d = dfres['feedback_avg_d1'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d1'].max()
rmin = dfres['feedback_avg_d1'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_feedback_avg_d1 = digital_one_hot_encoding(dfres, 'feedback_avg_d1')
dfres = dfres.drop('feedback_avg_d1', axis=1)

In [ ]:
d = dfres['feedback_avg_d2'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d2'].max()
rmin = dfres['feedback_avg_d2'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_feedback_avg_d2 = digital_one_hot_encoding(dfres, 'feedback_avg_d2')
dfres = dfres.drop('feedback_avg_d2', axis=1)

In [ ]:
d = dfres['feedback_avg_d3'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d3'].max()
rmin = dfres['feedback_avg_d3'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_feedback_avg_d3 = digital_one_hot_encoding(dfres, 'feedback_avg_d3')
dfres = dfres.drop('feedback_avg_d3', axis=1)

In [ ]:
d = dfres['feedback_avg_d4'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d4'].max()
rmin = dfres['feedback_avg_d4'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_feedback_avg_d4 = digital_one_hot_encoding(dfres, 'feedback_avg_d4')
dfres = dfres.drop('feedback_avg_d4', axis=1)

In [ ]:
d = dfres['feedback_avg_d5'].value_counts()
l = len(d)
rmax = dfres['feedback_avg_d5'].max()
rmin = dfres['feedback_avg_d5'].min()
d, l, rmax, rmin

In [ ]:
dfres, codes_feedback_avg_d5 = digital_one_hot_encoding(dfres, 'feedback_avg_d5')
dfres = dfres.drop('feedback_avg_d5', axis=1)

In [ ]:
dfres.to_csv("RSF_dfres.csv", index=True)

In [ ]:
dfres.info()

In [ ]:
dfres, codes_target = simple_one_hot_encoding(dfres, 'target')

In [ ]:
dfres = dfres.drop('target', axis=1)

In [ ]:
dfres.head()

### Разбиваем обратно набор на обучающий и тестовый

In [ ]:
def split_back(df, len_df):
    #features = df.iloc[:, 0:5] + df.iloc[:,11:]
    df1 = df.iloc[:len_df, :]
    df2 = df.iloc[len_df:, :]
    return df1, df2

In [ ]:
data_df, test_df = split_back(dfres, len_df)